# Silver layer

In [ ]:
in_parameter_run_id = 0
in_parameter_process_date = ""
in_parameter_table_name = ""
in_parameter_is_fact_table = True
in_parameter_year = 2010
out_parameter_count_processed = 0

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
import pandas as pd

In [ ]:
df_logs = pd.DataFrame(columns=["run_id", "code", "description"])

In [ ]:
v_message = ""
v_notebook_name = f"04_notebook_{in_parameter_table_name}_silver_layer"
try:
    if in_parameter_is_fact_table == True:
        out_parameter_count_processed = mssparkutils.notebook.run(v_notebook_name, timeout_seconds = 30, arguments = {
            "in_parameter_run_id": in_parameter_run_id,
            "in_parameter_process_date": in_parameter_process_date,
            "in_parameter_year": in_parameter_year
        })
    else:
        out_parameter_count_processed = mssparkutils.notebook.run(v_notebook_name, timeout_seconds = 30, arguments = {
            "in_parameter_run_id": in_parameter_run_id,
            "in_parameter_process_date": in_parameter_process_date
        })
    df_logs.loc[0] = [in_parameter_run_id, "02", f"{v_notebook_name}"]
except Exception as e:
    v_message = str(e)
    df_logs.loc[0] = [in_parameter_run_id, "01", f"{v_notebook_name} - {v_message}"]

In [ ]:
df_logs = spark.createDataFrame(df_logs)
df_logs = df_logs.withColumn("run_id", col("run_id").cast(IntegerType()))
df_logs.write.format("delta").mode("append").saveAsTable("control.logs")

In [ ]:
mssparkutils.notebook.exit(out_parameter_count_processed)